In [8]:
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
import requests
from census import Census
from pprint import pprint

from config import api_key
c = Census(api_key)

In [66]:
#loading in raw data CSVs for cleaning (crime data not cleaned here)
crime_2011 = pd.read_csv('2011_analysis_data.csv')
crime_2015 = pd.read_csv('2015_analysis_data.csv')
crime_2021 = pd.read_csv('2021_analysis_data.csv')
school_2011 = pd.read_csv('chi_school_2011_2012.csv')
school_2015 = pd.read_csv('chi_school_2015_2016.csv')
school_2021 = pd.read_csv('chi_school_2021_2022.csv')

FileNotFoundError: [Errno 2] No such file or directory: '2011_analysis_data.csv'

In [11]:
# Adding years to the school data frames for separation when merged
school_2011['Year'] = 2011
school_2015['Year'] = 2015
school_2021['Year'] = 2021
years = [2021, 2015, 2011]

In [ ]:
# Concatenating Crime DFs together (not used further here
crime_data = [crime_2011, crime_2015, crime_2021]
crime_df = pd.concat(crime_data)
crime_df['Ward'] = crime_df['Ward'].astype(int)
print(len(crime_df))

812281


In [31]:
# Concatenating and filtering school data for only desired columns
school_data = [school_2011, school_2015, school_2021]
school_df = pd.concat(school_data)
school_df_filtered = school_df[['School ID', 'School_ID', 'Year', 'Elementary, Middle, or High School', 'Primary_Category', 'ZIP Code', 'Zip',
                                'Student_Attendance_Avg_Pct', 'Average Student Attendance', 'Student_Attendance_Year_1_Pct',
                                'College Enrollment Rate %', 'College_Enrollment_School_Pct_Year_1']]
school_df_filtered.head()

,School ID,School_ID,Year,"Elementary, Middle, or High School",Primary_Category,ZIP Code,Zip,Student_Attendance_Avg_Pct,Average Student Attendance,Student_Attendance_Year_1_Pct,College Enrollment Rate %,College_Enrollment_School_Pct_Year_1
0,609966.0,NaN,2011,ES,NaN,60623.0,NaN,NaN,95.3,NaN,NDA,NaN
1,610539.0,NaN,2011,ES,NaN,60634.0,NaN,NaN,95.1,NaN,NDA,NaN
2,609852.0,NaN,2011,ES,NaN,60625.0,NaN,NaN,95.1,NaN,NDA,NaN
3,609835.0,NaN,2011,ES,NaN,60651.0,NaN,NaN,92.6,NaN,NDA,NaN
4,610521.0,NaN,2011,ES,NaN,60636.0,NaN,NaN,93.0,NaN,NDA,NaN


In [32]:
# Cleaning ES, MS, and HS distinctions to eventually filter out all ES and MS
school_df_filtered['Elementary, Middle, or High School'].fillna(school_df_filtered['Primary_Category'], inplace=True)
print(school_df_filtered['Elementary, Middle, or High School'].isna().sum())
school_df_filtered['Elementary, Middle, or High School'] = school_df_filtered['Elementary, Middle, or High School'].astype(str)
del school_df_filtered['Primary_Category']

0


C:\Users\Billy\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/4072691438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_df_filtered['Elementary, Middle, or High School'] = school_df_filtered['Elementary, Middle, or High School'].astype(str)


In [33]:
hs_df = school_df_filtered.loc[school_df_filtered['Elementary, Middle, or High School'] == "HS", :]

In [34]:
# Checking for empty rows in college enrollment
hs_df['College Enrollment Rate %'].fillna(hs_df['College_Enrollment_School_Pct_Year_1'], inplace=True)
print(hs_df['College Enrollment Rate %'].isna().sum())

57


In [35]:
# Combining columns and removing NaN for college enrollment
hs_df['College Enrollment Rate %'] = hs_df['College Enrollment Rate %'].replace('NDA', 57.6)
hs_df['College Enrollment Rate %'] = hs_df['College Enrollment Rate %'].astype(float)
hs_df['College Enrollment Rate %'] = hs_df['College Enrollment Rate %'].fillna(57.6)
del hs_df['College_Enrollment_School_Pct_Year_1']
print(hs_df['College Enrollment Rate %'].isna().sum())

0


C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/2208931989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['College Enrollment Rate %'] = hs_df['College Enrollment Rate %'].replace('NDA', 57.6)
C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/2208931989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['College Enrollment Rate %'] = hs_df['College Enrollment Rate %'].astype(float)
C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/2208931989.py:3: SettingWithCopyWarning: 
A value is trying 

In [36]:
# Combining columns and removing NaN for school ID
hs_df['School ID'].fillna(hs_df['School_ID'], inplace=True)
print(hs_df['School ID'].isna().sum())
hs_df['School ID'] = hs_df['School ID'].astype(int)
del hs_df['School_ID']

0


C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/2553016964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['School ID'] = hs_df['School ID'].astype(int)


In [37]:
# Combining columns and removing NaN for zip code
hs_df['ZIP Code'].fillna(hs_df['Zip'], inplace=True)
print(hs_df['ZIP Code'].isna().sum())
hs_df['ZIP Code'] = hs_df['ZIP Code'].astype(int)
del hs_df['Zip']

0


C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/2823837639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['ZIP Code'] = hs_df['ZIP Code'].astype(int)


In [38]:
# Combining columns and removing NaN for attendance
hs_df['Average Student Attendance'].fillna(hs_df['Student_Attendance_Avg_Pct'], inplace=True)
hs_df['Average Student Attendance'].fillna(hs_df['Student_Attendance_Year_1_Pct'], inplace=True)
hs_df['Average Student Attendance'].fillna(86.09, inplace=True)
hs_df['Average Student Attendace'] = hs_df['Average Student Attendance'].astype(float)
del hs_df['Student_Attendance_Avg_Pct']
del hs_df['Student_Attendance_Year_1_Pct']
print(hs_df['Average Student Attendance'].isna().sum())

0


C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/2657816923.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['Average Student Attendace'] = hs_df['Average Student Attendance'].astype(float)


In [41]:
hs_df

,School ID,Year,"Elementary, Middle, or High School",ZIP Code,Average Student Attendance,College Enrollment Rate %,Average Student Attendace
8,609680,2011,HS,60610,93.4,82.4,93.4
15,609722,2011,HS,60612,66.8,51.9,66.8
17,609749,2011,HS,60625,95.7,90.7,95.7
28,610244,2011,HS,60644,79.1,60.4,79.1
30,610394,2011,HS,60640,88.7,65.0,88.7
...,...,...,...,...,...,...,...
641,609698,2021,HS,60652,85.9,53.5,85.9
642,609748,2021,HS,60608,73.7,4.0,73.7
646,400126,2021,HS,60651,59.7,21.1,59.7
651,400119,2021,HS,60624,85.5,48.9,85.5


In [44]:
# Finding unique zip codes for pulling population data
zip_codes = hs_df['ZIP Code'].unique()
print(len(zip_codes))
zip_codes

46


array([60610, 60612, 60625, 60644, 60640, 60643, 60615, 60652, 60619,
       60623, 60632, 60617, 60636, 60608, 60637, 60624, 60653, 60620,
       60629, 60641, 60616, 60634, 60618, 60660, 60613, 60621, 60628,
       60627, 60631, 60622, 60614, 60639, 60609, 60607, 60605, 60659,
       60626, 60638, 60655, 60649, 60651, 60602, 60642, 60647, 60661,
       60827])

In [65]:
# Runs through the Census API from class exercise to fill in population data for 2011 and 2015
zipcode_pop = []
zipcode_year = []
zipcode_zipcode = []
counter = 1
for yr in years:
    for zipcode in zip_codes:
        
        try:
            c = Census(api_key, year=yr)
    
            census_data = c.acs5.get(("NAME", "B19013_001E"), 
                                         geo={'for': f"zip code tabulation area:{zipcode}",
                                           'in': 'state:17'
                                        })
   # print(row['Year'])
   # print(row['ZIP Code'])
            print(counter, census_data[0]['B19013_001E'])
            zipcode_pop.append(census_data[0]['B19013_001E'])
            zipcode_year.append(yr)
            zipcode_zipcode.append(zipcode)
        except:
            print(counter, "No Value")
            zipcode_pop.append("")
            zipcode_year.append(yr)
            zipcode_zipcode.append(zipcode)
        
        counter = counter + 1

1 No Value
2 No Value
3 No Value
4 No Value
5 No Value
6 No Value
7 No Value
8 No Value
9 No Value
10 No Value
11 No Value
12 No Value
13 No Value
14 No Value
15 No Value
16 No Value
17 No Value
18 No Value
19 No Value
20 No Value
21 No Value
22 No Value
23 No Value
24 No Value
25 No Value
26 No Value
27 No Value
28 No Value
29 No Value
30 No Value
31 No Value
32 No Value
33 No Value
34 No Value
35 No Value
36 No Value
37 No Value
38 No Value
39 No Value
40 No Value
41 No Value
42 No Value
43 No Value
44 No Value
45 No Value
46 No Value
47 75892.0
48 No Value
49 No Value
50 No Value
51 No Value
52 59593.0
53 No Value
54 61797.0
55 No Value
56 No Value
57 No Value
58 37796.0
59 27871.0
60 36216.0
61 26845.0
62 No Value
63 25923.0
64 No Value
65 No Value
66 51597.0
67 No Value
68 56382.0
69 57500.0
70 No Value
71 72126.0
72 No Value
73 No Value
74 No Value
75 76400.0
76 75163.0
77 92714.0
78 No Value
79 32284.0
80 85917.0
81 No Value
82 48104.0
83 No Value
84 62511.0
85 No Value
86 No Va

In [46]:
# Creates a dataframe for zip code data.
# Also fills any NA values in 2011 and 2015 with the years avg population
zipcode_data_df = pd.DataFrame({'Zipcode': zipcode_zipcode,
                                'Year': zipcode_year,
                                'Population': zipcode_pop})
without_blank = zipcode_data_df.loc[zipcode_data_df['Population'] != "", :]
without_blank['Population'] = without_blank['Population'].astype(int)
mean_2011 = round(without_blank.loc[without_blank['Year'] == 2011, :]['Population'].mean(), 0)
mean_2015 = round(without_blank.loc[without_blank['Year'] == 2015, :]['Population'].mean(), 0)
print(mean_2011, mean_2015)

50556.0 50407.0


C:\Users\Billy\AppData\Local\Temp/ipykernel_5016/1905830749.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  without_blank['Population'] = without_blank['Population'].astype(int)


In [47]:
# Fills values in DF with mean value
for idx, row in zipcode_data_df.iterrows():
    if (row[1] == 2011) & (row[2] == ""):
        zipcode_data_df.loc[idx, 'Population'] = mean_2011
    elif (row[1] == 2015) & (row[2] == ""):
        zipcode_data_df.loc[idx, 'Population'] = mean_2015

In [48]:
zipcode_data_df

,Zipcode,Year,Population
0,60610,2021,
1,60612,2021,
2,60625,2021,
3,60644,2021,
4,60640,2021,
...,...,...,...
133,60602,2011,50556.0
134,60642,2011,65463.0
135,60647,2011,50297.0
136,60661,2011,86894.0


In [49]:
# Calculates growth rates from 2011 to 2015 and adds it to a dictionary for reference later
pop_growth = {}
for zipcode in zip_codes:
    for idx, row in zipcode_data_df.iterrows():
        try:
            if (row['Year'] == 2011) & (row['Zipcode'] == zipcode):
                pop_2011 = row[2]
               # print(row[2])
            elif (row['Year'] == 2015) & (row['Zipcode'] == zipcode):
                pop_2015 = row[2]
               # print(row[2])
            growth_rate = round((pop_2015 - pop_2011) / pop_2011, 2)
        except:
            growth_rate = 0
    #print(growth_rate)
    pop_growth[zipcode] = growth_rate
print(pop_growth)


{60610: 0.08, 60612: 0.02, 60625: 0.12, 60644: -0.0, 60640: -0.0, 60643: 0.18, 60615: 0.3, 60652: 0, 60619: -0.0, 60623: -0.0, 60632: -0.24, 60617: -0.03, 60636: -0.45, 60608: 0.42, 60637: -0.47, 60624: 1.19, 60653: 1.0, 60620: -0.09, 60629: -0.02, 60641: 0.02, 60616: 0.17, 60634: -0.0, 60618: 0.0, 60660: 0.07, 60613: 0.08, 60621: -0.61, 60628: -0.28, 60627: -0.0, 60631: -0.0, 60622: -0.21, 60614: -0.0, 60639: -0.0, 60609: -0.0, 60607: 0.7, 60605: 0.73, 60659: 0.04, 60626: -0.08, 60638: -0.17, 60655: 0.02, 60649: -0.47, 60651: -0.0, 60602: 0.13, 60642: -0.23, 60647: 0.12, 60661: -0.42, 60827: -0.0}


In [50]:
df_2015 = zipcode_data_df.loc[zipcode_data_df['Year'] == 2015, :]

In [53]:
# Adds 2021 population using growth rates to a dictionary to add to DF
population_2021 = {}
for idx, row in df_2015.iterrows():
    pop_2021 = row[2] * (1 + pop_growth[row[0]])
    population_2021[row[0]] = pop_2021

In [54]:
zipcode_data_df.head()

,Zipcode,Year,Population
0,60610,2021,
1,60612,2021,
2,60625,2021,
3,60644,2021,
4,60640,2021,


In [55]:
# Adds population for 2021 to DF by zipcode
for idx, row in zipcode_data_df.iterrows():
    if row[2] == "":
        zipcode_data_df.loc[idx, 'Population'] = population_2021[row[0]]
zipcode_data_df

,Zipcode,Year,Population
0,60610,2021,81963.36
1,60612,2021,36605.76
2,60625,2021,63287.84
3,60644,2021,50407.0
4,60640,2021,50407.0
...,...,...,...
133,60602,2011,50556.0
134,60642,2011,65463.0
135,60647,2011,50297.0
136,60661,2011,86894.0


In [56]:
# Merges population back into the original DF
hs_school_df = pd.merge(hs_df, zipcode_data_df, how='left', left_on='ZIP Code', right_on='Zipcode')
hs_school_df

,School ID,Year_x,"Elementary, Middle, or High School",ZIP Code,Average Student Attendance,College Enrollment Rate %,Average Student Attendace,Zipcode,Year_y,Population
0,609680,2011,HS,60610,93.4,82.4,93.4,60610,2021,81963.36
1,609680,2011,HS,60610,93.4,82.4,93.4,60610,2015,75892.0
2,609680,2011,HS,60610,93.4,82.4,93.4,60610,2011,70268.0
3,609722,2011,HS,60612,66.8,51.9,66.8,60612,2021,36605.76
4,609722,2011,HS,60612,66.8,51.9,66.8,60612,2015,35888.0
...,...,...,...,...,...,...,...,...,...,...
1363,400119,2021,HS,60624,85.5,48.9,85.5,60624,2015,50407.0
1364,400119,2021,HS,60624,85.5,48.9,85.5,60624,2011,22982.0
1365,609760,2021,HS,60827,92.4,64.0,92.4,60827,2021,50407.0
1366,609760,2021,HS,60827,92.4,64.0,92.4,60827,2015,50407.0


In [57]:
# Filters out the rows where the year isn't the same
hs_school_df = hs_school_df.loc[hs_school_df['Year_x'] == hs_school_df['Year_y'], :]
hs_school_df

,School ID,Year_x,"Elementary, Middle, or High School",ZIP Code,Average Student Attendance,College Enrollment Rate %,Average Student Attendace,Zipcode,Year_y,Population
2,609680,2011,HS,60610,93.4,82.4,93.4,60610,2011,70268.0
5,609722,2011,HS,60612,66.8,51.9,66.8,60612,2011,35013.0
8,609749,2011,HS,60625,95.7,90.7,95.7,60625,2011,50556.0
11,610244,2011,HS,60644,79.1,60.4,79.1,60644,2011,50556.0
14,610394,2011,HS,60640,88.7,65.0,88.7,60640,2011,50556.0
...,...,...,...,...,...,...,...,...,...,...
1353,609698,2021,HS,60652,85.9,53.5,85.9,60652,2021,61797
1356,609748,2021,HS,60608,73.7,4.0,73.7,60608,2021,71577.94
1359,400126,2021,HS,60651,59.7,21.1,59.7,60651,2021,50407.0
1362,400119,2021,HS,60624,85.5,48.9,85.5,60624,2021,110391.33


In [58]:
# Remove unnecessary columns for export
school_data_df = hs_school_df[['School ID', 'Year_x', 'Elementary, Middle, or High School', 'ZIP Code',
                               'Average Student Attendance', 'College Enrollment Rate %', 'Population']]
school_data_df

,School ID,Year_x,"Elementary, Middle, or High School",ZIP Code,Average Student Attendance,College Enrollment Rate %,Population
2,609680,2011,HS,60610,93.4,82.4,70268.0
5,609722,2011,HS,60612,66.8,51.9,35013.0
8,609749,2011,HS,60625,95.7,90.7,50556.0
11,610244,2011,HS,60644,79.1,60.4,50556.0
14,610394,2011,HS,60640,88.7,65.0,50556.0
...,...,...,...,...,...,...,...
1353,609698,2021,HS,60652,85.9,53.5,61797
1356,609748,2021,HS,60608,73.7,4.0,71577.94
1359,400126,2021,HS,60651,59.7,21.1,50407.0
1362,400119,2021,HS,60624,85.5,48.9,110391.33


In [ ]:
# Export data to a csv for further analysis
school_data_df.to_csv('cleaned_school_data.csv')

In [59]:
school_data_df.groupby('Year_x').count()

,School ID,"Elementary, Middle, or High School",ZIP Code,Average Student Attendance,College Enrollment Rate %,Population
Year_x,,,,,,
2011,93,93,93,93,93,93
2015,188,188,188,188,188,188
2021,175,175,175,175,175,175


In [60]:
# Separate data out for each year to create plots
school_data_df = pd.read_csv('cleaned_school_data.csv')
school_2011_df = school_data_df[school_data_df['Year_x'] == 2011]
school_2015_df = school_data_df[school_data_df['Year_x'] == 2015]
school_2021_df = school_data_df[school_data_df['Year_x'] == 2021]



In [61]:
school_2011_df.head()

,Unnamed: 0,School ID,Year_x,"Elementary, Middle, or High School",ZIP Code,Average Student Attendance,College Enrollment Rate %,Population
0,2,609680,2011,HS,60610,93.4,82.4,70268.0
1,5,609722,2011,HS,60612,66.8,51.9,35013.0
2,8,609749,2011,HS,60625,95.7,90.7,53233.0
3,11,610244,2011,HS,60644,79.1,60.4,28235.0
4,14,610394,2011,HS,60640,88.7,65.0,44964.0


1. for each year average by zip code for enrollment rate and attendance

In [62]:
# Aggregates data to get mean of school target variables
grouped_zips_2011_df = school_2011_df.groupby('ZIP Code').mean()
grouped_zips_2011_df.reset_index(inplace=True)
grouped_zips_2011_df = grouped_zips_2011_df[['ZIP Code', 'Year_x',
       'Average Student Attendance', 'College Enrollment Rate %',
       'Population']]
grouped_zips_2011_df.head()

,ZIP Code,Year_x,Average Student Attendance,College Enrollment Rate %,Population
0,60605,2011.0,93.800000,86.9,75030.0
1,60607,2011.0,94.600000,88.3,79242.0
2,60608,2011.0,83.700000,48.8,35477.0
3,60609,2011.0,87.433333,45.9,34725.0
4,60610,2011.0,93.400000,82.4,70268.0


In [63]:
# Aggregates data to get mean of school target variables
grouped_zips_2015_df = school_2015_df.groupby('ZIP Code').mean()
grouped_zips_2015_df.reset_index(inplace=True)
grouped_zips_2015_df = grouped_zips_2015_df[['ZIP Code', 'Year_x',
       'Average Student Attendance', 'College Enrollment Rate %',
       'Population']]
grouped_zips_2015_df.head()

,ZIP Code,Year_x,Average Student Attendance,College Enrollment Rate %,Population
0,60602,2015.0,88.8,59.550000,57368.0
1,60605,2015.0,88.8,88.500000,87668.0
2,60607,2015.0,88.8,79.300000,85917.0
3,60608,2015.0,88.8,44.272727,36216.0
4,60609,2015.0,88.8,40.628571,32284.0


In [64]:
# Aggregates data to get mean of school target variables
grouped_zips_2021_df = school_2021_df.groupby('ZIP Code').mean()
grouped_zips_2021_df.reset_index(inplace=True)
grouped_zips_2021_df = grouped_zips_2021_df[['ZIP Code', 'Year_x',
       'Average Student Attendance', 'College Enrollment Rate %',
       'Population']]
grouped_zips_2021_df.head()

,ZIP Code,Year_x,Average Student Attendance,College Enrollment Rate %,Population
0,60602,2021.0,72.7000,54.3500,33273.0
1,60605,2021.0,91.3000,86.4000,102572.0
2,60607,2021.0,90.3950,73.0000,92790.0
3,60608,2021.0,77.4625,43.9875,36940.0
4,60609,2021.0,85.8400,51.8800,30024.0
